We can define the LP as follows. $n$ is the length of the grid, and $s$ is the length of a sub-square within the grid. For instance, if $n = 9$, $s = 3$.

\begin{alignat*}{3}
\sum_{i, j}^{n} x_{ijv} = 1 \\
\sum_{i, v}^{n} x_{ijv} = 1 \\
\sum_{j, v}^{n} x_{ijv} = 1 \\
\sum_{i=si'}^{s(i'+1)} \sum_{j=sj'}^{s(j'+1)} x_{ijv} = 1 \\
i, j, v \in [1, n] \\
i', j' \in [1, s]
\end{alignat*}

In [1]:
import random
import copy
import gurobipy as gp
from gurobipy import GRB, quicksum, max_, abs_
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def create_empty_matrix(size=9):
    #0 = "empty"
    return np.array([[0 for x in range(size)] for y in range(size)])

In [3]:
def load_test():
    matrix = create_empty_matrix(9)

    matrix[0][1]=2
    matrix[0][4]=3
    matrix[0][7]=4
    matrix[1][0]=6
    matrix[1][8]=3
    matrix[2][2]=4
    matrix[2][6]=5
    matrix[3][3]=8
    matrix[3][5]=6
    matrix[4][0]=8
    matrix[4][4]=1
    matrix[4][8]=6
    matrix[5][3]=7
    matrix[5][5]=5
    matrix[6][2]=7
    matrix[6][6]=6
    matrix[7][0]=4
    matrix[7][8]=8
    matrix[8][1]=3
    matrix[8][4]=4
    matrix[8][7]=2

    return matrix

In [4]:
def print_solution(grid, model, variables):
    print('')
    print('Solution:')
    print('')
    n = len(grid[0])
    s = int(n**0.5)

    # Retrieve optimization result

    solution = model.getAttr('X', variables)

    for i in range(n):
        sol = ''
        if s != 0 and i % s == 0:
            sol += '\n'
        for j in range(n):
            if j != 0 and j % s == 0:
                sol += '  '
            for v in range(n):
                if solution[i, j, v] > 0.5:
                    sol += str(v+1)
        print(sol)

In [5]:
def gurobi_solution(grid):
# In the MIP formulation, binary variables x[i,j,v] indicate whether
# cell <i,j> takes value 'v'.  The constraints are as follows:
#   1. Each cell must take exactly one value (sum_v x[i,j,v] = 1)
#   2. Each value is used exactly once per row (sum_i x[i,j,v] = 1)
#   3. Each value is used exactly once per column (sum_j x[i,j,v] = 1)
#   4. Each value is used exactly once per 3x3 subgrid (sum_grid x[i,j,v] = 1)

    n = len(grid[0])
    s = int(n**0.5)
    mdl = gp.Model('gurobi')
    # mdl.Params.LogToConsole = 0
    var = mdl.addVars(n, n, n, vtype=GRB.BINARY, name='G')


    # Fix variables associated with cells whose values are pre-specified
    for i in range(n):
        for j in range(n):
            if grid[i][j] > 0:
                v = int(grid[i][j]) - 1
                var[i, j, v].LB = 1


    # Each cell must take one value
    mdl.addConstrs((var.sum(i, j, '*') == 1
                     for i in range(n)
                     for j in range(n)), name='V')


    # Each value appears once per row
    mdl.addConstrs((var.sum(i, '*', v) == 1
                     for i in range(n)
                     for v in range(n)), name='R')


    # Each value appears once per column
    mdl.addConstrs((var.sum('*', j, v) == 1
                     for j in range(n)
                     for v in range(n)), name='C')


    # Each value appears once per subgrid
    mdl.addConstrs((
        gp.quicksum(var[i, j, v] for i in range(i0*s, (i0+1)*s)
                    for j in range(j0*s, (j0+1)*s)) == 1
        for v in range(n)
        for i0 in range(s)
        for j0 in range(s)), name='Sub')

    mdl.optimize()

    return mdl, var

In [6]:

grid = load_test()
mdl, var = gurobi_solution(grid)
print_solution(grid, mdl, var)

Using license file C:\Users\marka\gurobi.lic
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (win64)
Optimize a model with 324 rows, 729 columns and 2916 nonzeros
Model fingerprint: 0xe9ce958b
Variable types: 0 continuous, 729 integer (729 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 324 rows and 729 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds
Thread count was 1 (of 4 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%

Solution:


925  631  847
618  574  293
374  982  561

749  826  135
852  413  976
163  795  482

287  359  614
491  267  358
536  148  729


In [ ]:
def is_used(grid, row, col, num):
    if in_row(grid, row, num):
        return True
    elif in_col(grid, col, num):
        return True
    elif in_mini(grid, row, col, num):
        return True
    return False
    

def in_row(grid, row, num):
    return num in grid[row]


def in_col(grid, col, num):
    size = len(grid[0])
    col_list = [grid[r][col] for r in range(size)]
    return num in col_list


def get_mini_list(grid, row, col):
    size = len(grid[0])
    root = int(size**0.5)
    mini_row = root*(row // root)
    mini_row2 = mini_row + root
    mini_col = root*(col // root)
    mini_col2 = mini_col + root
    mini_list = [grid[r][c] for r in range(mini_row, mini_row2) for c in range(mini_col, mini_col2)]
    return mini_list


def in_mini(grid, row, col, num):
    return num in get_mini_list(grid, row, col)


def has_zeroes(grid):
    size = len(grid[0])
    for row in range(size):
        if 0 in grid[row]:
            return True
    return False

def count_occurances(grid, num=0):
    return sum(cell == num for row in grid for cell in row)

In [ ]:
def generate_grid(size=9):
    if(size**0.5 != int(size**0.5)):
        raise ValueError("Size must be a perfect square")
    #creates an "empty" grid
    grid = np.array([[0 for x in range(size)] for y in range(size)])
    fill_grid(grid)
    return grid


def fill_grid(grid, solve=False):
    global solver_counter
    size = len(grid[0])
    squared = size**2
    numbers = [temp+1 for temp in range(size)]
    for i in range(0, squared):
        row = i // size
        col = i % size
        if grid[row][col] != 0:
            continue
        if not solve:
            random.shuffle(numbers)
        for num in numbers:
            if not is_used(grid, row, col, num):
                grid[row][col] = num
                if not has_zeroes(grid):
                    solver_counter += 1
                    if solve:
                        break
                    return True
                elif fill_grid(grid, solve):
                    return True
        break
    grid[row][col] = 0


In [ ]:
def sparsify(grid, attempts=5):
    # larger number of attempts = the more sparse it gets
    # Warning this could go for awhile!
    global solver_counter
    size = len(grid[0])
    original = copy.deepcopy(grid)
    while attempts > 0:
        row = random.randint(0, size-1)
        col = random.randint(0, size-1)
        while grid[row][col] == 0:
            row = random.randint(0, size-1)
            col = random.randint(0, size-1)
        
        backup = grid[row][col]
        grid[row][col] = 0
        test_grid = copy.deepcopy(grid)

        solver_counter = 0
        fill_grid(test_grid, True)

        if solver_counter != 1:
            grid[row][col] = backup
            attempts -= 1
            print("DECREMENTING ATTEMPTS {} - solver_counter={}".format(attempts, solver_counter))
    return original, grid


def sparsify_no_check(grid, num_clues=None):
    size = len(grid[0])
    clues = num_clues or (size**2 // 4) # just a good speculation
    # 9x9 min = 17     (size**2 // 4 == 20)
    # 16x16 min ?= 51  (size**2 // 4 == 64)
    # 25x25 min ?= 151 (size**2 // 4 == 156)
    original = copy.deepcopy(grid)
    remaining = size**2
    while remaining > clues:
        row = random.randint(0, size-1)
        col = random.randint(0, size-1)
        while grid[row][col] == 0:
            row = random.randint(0, size-1)
            col = random.randint(0, size-1)
        grid[row][col] = 0
        remaining -= 1
    return original, grid